**<h1 align="center">PROYECTO INDIVIDUAL 2</h1>**
<h1 align="center"> Limpieza de Datos</h1>

Importamos librerias

In [7]:
import pandas as pd
import numpy as np
import re
import datetime

Abrimos el dataset y trabajamos las dos hojas de calculo que necesitamos en diferentes Dataframe

In [8]:
#Abres el libro de Excel 
xlsx = pd.ExcelFile('homicidios.xlsx')
#Usamos las hojas del libro de Excel que necesitamos en Dataframes diferentes 
df1 = pd.read_excel(xlsx, 'HECHOS')
df2 = pd.read_excel(xlsx, 'VICTIMAS')

Limpieza de Datos de la Hoja de calculo 'HECHOS' en el Dataframe df1

In [9]:
#Eliminamos columnas no relevantes para el analisis posterior
df1 = df1.drop('Cruce', axis= 1)
df1 = df1.drop('Altura', axis= 1)
df1 = df1.drop('Calle', axis= 1)
df1 = df1.drop('Dirección Normalizada', axis= 1)

#Convertimos el tipo de variable de datos de las siguientes columnas
df1['ID'] = df1['ID'].astype(str)
df1['LUGAR_DEL_HECHO'] = df1['LUGAR_DEL_HECHO'].astype(str)
df1['TIPO_DE_CALLE'] = df1['TIPO_DE_CALLE'].astype(str)
df1['VICTIMA'] = df1['VICTIMA'].astype(str)
df1['ACUSADO'] = df1['ACUSADO'].astype(str)

#Eliminamos las filas que tengan valor de "SD" en las columnas 'PARTICIPANTES', 'VICTIMA' y 'ACUSADO'
df1 = df1[(df1['PARTICIPANTES'] != 'SD-SD')]

#Reemplazamos el valor de "SD" de la columna 'HORA' la media de los datos de dicha columna
#Realizamos la funcion para intercambiar valores
def columns_modifier(mode,DF,columns,value,replace=None):
    df = DF.copy()
    if mode == 'swap':
        for index, row in df.iterrows():

            value_1=row[columns[0]]
            value_2=row[columns[1]]

            if value_1== value:
                temp=value_1
                df.at[index, columns[0]] = value_2
                df.at[index, columns[1]] = temp
    if mode == 'replace':
        for index, row in df.iterrows():
            to_replace=row[columns[0]]

            if to_replace == value:
                df.at[index, columns[0]] = replace
    return df

#Llamamos la funcion en las columnas 'HH' y 'HORA'
columna = ['HH']
df1 = columns_modifier('replace',df1,columna,'SD',np.nan)
columna = ['HORA']
df1 = columns_modifier('replace',df1,columna,'SD',np.nan)

#Realizamos la funcion para tomar un valor decimal y lo convierte en cadena de texto con formato de hora
def convertir_hora(decimal_hora):
    horas_enteras = int(decimal_hora)
    minutos_decimales = (decimal_hora - horas_enteras) * 60

    hora_formateada = f"{horas_enteras:02d}:{int(minutos_decimales):02d}"

    return hora_formateada

df1['HORA'] = df1['HORA'].fillna(convertir_hora(df1['HH'].mean()))
df1['HH'] = df1['HH'].fillna(df1['HH'].mean()).apply(int)

#Convertimos los valores de las siguientes columnas de Mayuscula a minuscula
df1['LUGAR_DEL_HECHO'] = df1['LUGAR_DEL_HECHO'].str.title()
df1['TIPO_DE_CALLE'] = df1['TIPO_DE_CALLE'].str.title()
df1['VICTIMA'] = df1['VICTIMA'].str.title()
df1['ACUSADO'] = df1['ACUSADO'].str.title()
df1['PARTICIPANTES'] = df1['PARTICIPANTES'].str.title()

#Reemplazamos el nombre de la columna 'ID' por el nombre 'ID_hecho'
df1 = df1.rename(columns={'ID': 'ID_hecho'})

#Guardamos Dataframe en un archivo .csv
df1.to_csv('Hechos.csv', index=False)

Limpieza de Datos de la Hoja de calculo 'VICTIMAS' en el Dataframe df2

In [10]:
#Realizamos el cambio de tipo de variable a las siguientes columnas
df2['ID_hecho'] = df2['ID_hecho'].astype(str)
df2['ROL'] = df2['ROL'].astype(str)
df2['VICTIMA'] = df2['VICTIMA'].astype(str)
df2['SEXO'] = df2['SEXO'].astype(str)

#Convertimos los valores de las siguientes columnas de Mayuscula a minuscula
df2['ROL'] = df2['ROL'].str.title()
df2['VICTIMA'] = df2['VICTIMA'].str.title()
df2['SEXO'] = df2['SEXO'].str.title()

#Debido a que hay valores en la columna'FECHA_FALLECIMIENTO' correspondiente a "SD" pero si comparamos con la columna 'FECHA' podemos observar que los valores son iguales, entonces para eliminar los valores "SD" asignamos los valores de la columna 'FECHA' a 'FECHA_FALLECIMIENTO'
df2['FECHA_FALLECIMIENTO'] = df2['FECHA'].where(df2['FECHA_FALLECIMIENTO'] == 'SD', df2['FECHA_FALLECIMIENTO'])

#Cambiamos el formato de la columna 'FECHA_FALLECIMIENTO' al formato de fecha correspondiente
df2['FECHA_FALLECIMIENTO'] = pd.to_datetime(df2['FECHA_FALLECIMIENTO'])

#Guardamos Dataframe en un archivo .csv
df2.to_csv('Victimas.csv', index=False)

Realizamos un merge entre los Dataframes df1 y df2

In [11]:
df3 = pd.merge(df1, df2, on=('ID_hecho', 'VICTIMA', 'FECHA', 'AAAA', 'MM', 'DD'), how='inner')

#Guardamos el merge en formato .csv
df3.to_csv("Homicidios.csv", index=False)